Introduction to Spark 02
====

Focus in this lecture is on Spark constructs that can make your programs more efficient. In general, this means minimizing the amount of data transfer across nodes, since this is usually the bottleneck for big data analysis problems.

- Shared variables
    - Accumulators
    - Broadcast variables
- DataFrames
- Partitioning and the Spark shuffle
- Piping to external programs

Spark tuning and optimization is complicated - this tutorial only touches on some of the basic concepts.

In [1]:
import numpy as np
import string

In [2]:
from pyspark import SparkContext
sc = SparkContext('local[*]')

Resources
----

[The Spark Programming Guide](http://spark.apache.org/docs/latest/programming-guide.html)

Accumulators
----

Spark functions such as `map` can use variables defined in the driver program, but they make local copies of the variable that are not passed back to the driver program. Accumulators are *shared variable* that allow the aggregation of results from workers back to the driver program, for example, as an event counter. Suppose we want to count the number of rows of data with missing information. The most efficient way is to use an **accumulator**.

In [3]:
ulysses = sc.textFile('data/Ulysses.txt')

In [4]:
ulysses.take(10)

['The Project Gutenberg EBook of Ulysses, by James Joyce',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org',
 '',
 '',
 'Title: Ulysses',
 '']

### Event counting

Notice that we have some empty lines. We want to count the number of non-empty lines.

In [5]:
num_lines = sc.accumulator(0)

def tokenize(line):
    table = dict.fromkeys(map(ord, string.punctuation))
    return line.translate(table).lower().strip().split()

def tokenize_count(line):
    global num_lines
    
    if line:
        num_lines += 1

    return tokenize(line)

In [6]:
counter = ulysses.flatMap(lambda line: tokenize_count(line)).countByValue()

In [7]:
counter['circle']

20

In [8]:
num_lines.value

25396

Broadcast Variables
----

Sometimes we need to send a large read only variable to all workers. For example, we might want to share a large feature matrix to all workers as a part of a machine learning application. This same variable will be sent separately for each parallel operation unless you use a **broadcast variable**. Also, the default variable passing mechanism is optimized for small variables and can be slow when the variable is large.

In [9]:
from itertools import count

table = dict(zip(string.ascii_letters, count()))

In [10]:
def weight_first(line, table):
    words = tokenize(line)
    return sum(table.get(word[0], 0) for word in words if word.isalpha())

def weight_last(line, table):
    words = tokenize(line)
    return sum(table.get(word[-1], 0) for word in words if word.isalpha())

#### The dictionary `table` is sent out twice to worker nodes, one for each call

In [11]:
ulysses.map(lambda line: weight_first(line, table)).sum()

2941855

In [12]:
ulysses.map(lambda line: weight_last(line, table)).sum()

2995994

#### Converting to use broadast variables is simple and more efficient

- Use SparkContext.broadcast() to create a broadcast variable
- Where you would use var, use var.value
- The broadcast variabel is sent once to each node and can be re-used

In [13]:
table_bc = sc.broadcast(table)

In [14]:
def weight_first_bc(line, table):
    words = tokenize(line)
    return sum(table.value.get(word[0], 0) for word in words if word.isalpha())

def weight_last_bc(line, table):
    words = tokenize(line)
    return sum(table.value.get(word[-1], 0) for word in words if word.isalpha())

#### table_bc is sent to nodes only once.

In [15]:
ulysses.map(lambda line: weight_first_bc(line, table_bc)).sum()

2941855

In [16]:
ulysses.map(lambda line: weight_last_bc(line, table_bc)).sum()

2995994

DataFrames
----

Spark has a DataFrame data structure that provides an interface for working with mixed type data sets, greatly simplifying distributed statistical data processing. DataFrames also allow easy conversion and merging of data from different sources (e.g. distributed databases or filesystem storage formats) and are highly optimized.

**Reminder** As usual - if your data set fits into memory, just use `pandas` - Spark is unlikely to provide any performance benefits unless your data set is truly massive.

In [17]:
import pandas as pd
from pyspark.sql import SQLContext

sql = SQLContext(sc)

In [18]:
url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv'
titanic = pd.read_csv(url)

In [19]:
df = sql.createDataFrame(titanic)
df

DataFrame[survived: bigint, pclass: bigint, sex: string, age: double, sibsp: bigint, parch: bigint, fare: double, embarked: string, class: string, who: string, adult_male: boolean, deck: double, embark_town: string, alive: string, alone: boolean]

In [20]:
df.head()

Row(survived=0, pclass=3, sex='male', age=22.0, sibsp=1, parch=0, fare=7.25, embarked='S', class='Third', who='man', adult_male=True, deck=nan, embark_town='Southampton', alive='no', alone=False)

#### Functional programming constructs still work

In [21]:
(
df.select('embark_town')
.map(lambda town: (town, 1))
.reduceByKey(lambda x, y: x+y)
.collect()
)

[(Row(embark_town='Queenstown'), 77),
 (Row(embark_town='NaN'), 2),
 (Row(embark_town='Southampton'), 644),
 (Row(embark_town='Cherbourg'), 168)]

#### You can use familiar methods like `groupby`

In [22]:
counts = df.groupby(['sex', 'class']).count()

In [23]:
counts.collect()

[Row(sex='female', class='First', count=94),
 Row(sex='female', class='Third', count=144),
 Row(sex='male', class='Second', count=108),
 Row(sex='female', class='Second', count=76),
 Row(sex='male', class='First', count=122),
 Row(sex='male', class='Third', count=347)]

#### We can convert from Spark to Pandas dataframes

In [24]:
counts.toPandas()

,sex,class,count
0,female,First,94
1,female,Third,144
2,male,Second,108
3,female,Second,76
4,male,First,122
5,male,Third,347


The Spark Shuffle and Partitioning
----

Some events trigger the redistribution of data across partitions, and involves the (expensive) copying of data across executors and machines. This is known as the **shuffle**. For example, if we do a `reduceByKey` operation on key-value pair RDD, Spark needs to collect all pairs with the same key in the same partition to do the reduction. 

For key-value RDDs, you have some control over the partitioning of the RDDs. In particular, you can ask Spark to partition a set of keys so that they are guaranteed to appear together on some node. This can minimize a lot of data transfer. For example, suppose you have a large key-value RDD consisting of user_name: comments from a web user community. Every night, you want to update with new user comments with a join operation

In [25]:
def fake_data(n, val):
    users = list(map(''.join, np.random.choice(list(string.ascii_lowercase), (n,2))))
    comments = [val]*n
    return tuple(zip(users, comments))

In [26]:
data = fake_data(10000, 0)
list(data)[:10]

[('nk', 0),
 ('pq', 0),
 ('ly', 0),
 ('pq', 0),
 ('oo', 0),
 ('lw', 0),
 ('cw', 0),
 ('au', 0),
 ('ce', 0),
 ('pt', 0)]

In [27]:
rdd = sc.parallelize(data).reduceByKey(lambda x, y: x+y)

In [28]:
new_data = fake_data(100, 1)
list(new_data)[:10]

[('xf', 1),
 ('xz', 1),
 ('ke', 1),
 ('au', 1),
 ('uq', 1),
 ('vi', 1),
 ('lu', 1),
 ('pv', 1),
 ('nc', 1),
 ('xt', 1)]

In [29]:
rdd_new = sc.parallelize(new_data).reduceByKey(lambda x, y: x+y).cache()

In [30]:
rdd_updated = rdd.join(rdd_new)

In [31]:
rdd_updated.take(10)

[('xf', (0, 1)),
 ('rx', (0, 1)),
 ('wf', (0, 1)),
 ('vo', (0, 1)),
 ('cb', (0, 2)),
 ('au', (0, 1)),
 ('hp', (0, 1)),
 ('ar', (0, 2)),
 ('oq', (0, 1)),
 ('gf', (0, 1))]

### Using `partitionBy`

The `join` operation will hash all the keys of both `rdd` and `rdd_nerw`, sending keys with the same hashes to the same node for the actual join operation. There is a lot of unnecessary data transfer. Since `rdd` is a much larger data set than `rdd_new`, we can instead fix the partitioning of `rdd` and just transfer the keys of `rdd_new`. This is done by `rdd.partitionBy(numPartitions)` where `numPartitions` should be at least twice the number of cores.

In [32]:
rdd2 = sc.parallelize(data).reduceByKey(lambda x, y: x+y)
rdd2 = rdd2.partitionBy(10).cache()

In [33]:
rdd2_updated = rdd2.join(rdd_new)

In [34]:
rdd2_updated.take(10)

[('zs', (0, 1)),
 ('hp', (0, 1)),
 ('wq', (0, 1)),
 ('vf', (0, 1)),
 ('xf', (0, 1)),
 ('rd', (0, 1)),
 ('nc', (0, 1)),
 ('dy', (0, 1)),
 ('dz', (0, 1)),
 ('zl', (0, 1))]

Piping to External Programs
----

Suppose it is more convenient or efficient to write a function in some other language to process data. We can **pipe** data from Spark to the external program (script) that performs the calculation via standard input and output. The example below shows using a C++ program to calculate the sum of squares for collections of numbers.

In [35]:
%%file foo.cpp

#include <iostream>
#include <sstream>
#include <string>
#include <numeric>
#include <vector>
using namespace std;

double sum_squares(double x, double y) {
  return x + y*y;
};

int main() {

    string s;

    while (cin) {

        getline(cin, s);
        stringstream stream(s);
        vector<double> v;

        while(1) {
            double u;
            stream >> u;
            if(!stream)
                break;
            v.push_back(u);
        }
        double x = accumulate(v.begin(), v.end(), 0.0, sum_squares);
        cout << x << endl;
    }
}

Overwriting foo.cpp


In [36]:
! g++ foo.cpp -o foo

In [37]:
xs = np.random.random((100, 3))
np.savetxt('numbers.txt', xs)

In [38]:
!head numbers.txt

8.007290252044240164e-01 9.388840172344149471e-01 7.641926913121750431e-01
4.797444598292510687e-02 6.371952930746385135e-01 1.728628973849547501e-01
7.368128692587038175e-01 9.560134392124460812e-01 1.520119012528126090e-01
6.138786203057124968e-01 2.304246773308167295e-01 7.612600522864221286e-01
5.858370203064489079e-02 9.319799159225684582e-01 8.741906149418643412e-02
5.974674822676994124e-01 2.354333172028766352e-01 2.251495926817642657e-01
5.743565883133471273e-01 7.083800554857178078e-01 4.511509816992009814e-01
1.626631002782216173e-01 1.591666834563804089e-01 3.510692468417206946e-01
6.374886282436751372e-01 5.362041285754336473e-01 6.951278106461864503e-01
9.591589607284622954e-01 7.159018076623913096e-02 4.989118457521155570e-01


In [39]:
rdd = sc.textFile('numbers.txt')

In [40]:
from pyspark import SparkFiles

def prepare(line):
    """Each line contains numbers separated by a space."""
    return ' '.join(line.split()) + '\n'

# pipe data to external function
func = './foo'
sc.addFile(func)
ss = rdd.map(lambda s: prepare(s)).pipe(SparkFiles.get(func))

In [41]:
ss.take(3)

['2.10666', '0.438201', '1.47996']

In [42]:
np.sum(xs[:3]**2, 1)

array([ 2.10666064,  0.43820097,  1.47996252])

### Version

In [43]:
%load_ext version_information
%version_information pyspark, numpy

Software versions
Python 3.5.1 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 4.0.1
OS Linux 4.2.0 23 generic x86_64 with debian jessie sid
pyspark The 'pyspark' distribution was not found and is required by the application
numpy 1.10.2
Mon Jan 18 19:44:12 2016 EST